# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

## Introduction

[//]: <> (Student provides a high-level overview of the project. Background information such as the problem domain, the project origin, and related data sets or input data is provided.)

This post discusses in detail a solution for the project "Create a Customer Segmentation Report for Arvato Financial Services", which is one of options for the Data Scientist Nanodegree capstone projects. It is in fact a continuation of a previous project whose solution was posted [here](https://github.com/bvcmartins/dsndProject3). I chose it because of its broad scope which involves a reasonably complex data cleaning procedure, unsupervised learning, analysis of imbalanced data, and prediction using supervised learning tools. In the following I will discuss the solution and the difficulties faced in some of the most challenging steps.

[//]: <> (The problem which needs to be solved is clearly defined. A strategy for solving the problem, including discussion of the expected solution, has been made.)

The objective of this project is to predict, based on a broad list of features, if someone who is receiving a mailout campaign will become a customer of the company. It is devided in three parts:

1. Segment the data obtained for the general population of Germany in clusters and identify in which of these clusters the current customers fall in
* Select people from these clusters who are not customers to receive a mailout campaign and train a supervised learning model to predict who will become a new customer
* Apply the model to a Kaggle competition

In part 1 I carried out the data segmentation using PCA for dimensionality reduction and K-Means for clustering. In part 2 I used three tree-based supervised learning techniques (Random Forests, Ada Boost, XGBoost) models for the prediction of new customers. 

[//]: <> (Metrics used to measure performance of a model or result are clearly defined. Metrics are justified based on the characteristics of the problem.)

The best model was tested against the test set through a Kaggle competition and the scores were ranked using the AUC (Area under curve) metric. This metric was used because of the highly imabalanced distribution of the response variable. However, in the last part of this report, I will show why this metric is not the best choice for this problem and why the F1 metric would be more suited to measure model performance.

### The dataset

[//]:<> (Features and calculated statistics relevant to the problem have been reported and discussed related to the dataset, and a thorough description of the input space or input data has been made. Abnormalities or characteristics about the data or input that need to be addressed have been identified.)

Arvato kindly provided us four datasets:

   * Azdias: sample of the general german population categorized according to a variety of features involving personality traits, demographics and financial information (891221 entries, 366 features).
   
   * Customers: classification of current customers according to the same features used for Azdias (191652 entries, 369 features). It also contains customer categorization and information about purchase preferences.
   
   * Mailout_train: training set containing potential customers chosen to receive a mailout campaign  (42962 entries, 367 features). It also contains information if the person became a customer (target variable).
   
   * Mailout_test: testing set for the supervised learning model (42833 entries, 366 features).

Two support files for the interpretation of the features were also provided:

   * DIAS Attributes - Values 2017: information about code levels for some of the features.

   * DIAS Information Levels - Attributes 2017: high-level information about most (but not all) features.

Most of the features are ordinal and the numbers represent a label for ranked value levels. As far as I could identify, all columns marked as float are actually of type int64 mixed with NaN symbols. In fact, NaNs are treated as float64 and, because of its higher precedence, they make the entire column being cast as float. Most recent Pandas versions (higher than 0.24.0) incorporated the new type Int64 which supports integer NaN. Despite being somewhat experimental, I used this new data type as a way to reduce the large amount of ancillary code required to control operations over different dtypes.

There are also 6 features of type object. These are categorical variables, except for EINGEFUEGT_AM, which is datetime. I dropped this feature because it did not have any explicative power.

Most of the columns contained NaNs. Actually, NaNs comprised almost 10% of all data. 

### Data cleaning and processing

[//]:<> (Build data visualizations to further convey the information associated with your data exploration journey. Ensure that visualizations are appropriate for the data values you are plotting.)

[//]:<> (All preprocessing steps have been clearly documented. Abnormalities or characteristics about the data or input that needed to be addressed have been corrected. If no data preprocessing is necessary, it has been clearly justified.)

Cleaning this dataset was a relatively complex task mainly because of the number and diversity of features. The steps are outlined below:

* pre-cleaning
* converting missing values to NaN
* assessing missing data per feature
* assessing missing data per row
* imputing missing data
* re-encoding mixed features
* one-hot encoding categorical features
* scaling numerical features
* removing outliers

[//]:<> (The process for which metrics, algorithms, and techniques were implemented with the given datasets or input data has been thoroughly documented. Complications that occurred during the coding process are discussed.)

[//]:<> (The process of improving upon the algorithms and techniques used is clearly documented. Both the initial and final solutions are reported, along with intermediate solutions, if necessary.)

#### Pre-cleaning

In this first cleaning round I applied general-purpose operations like converting all numeric features to Int64 (support to integer NaN) and made substitutions of some non-standard missing data codes.

#### Converting missing values to NaN

The challenge with this step was that the missing data codes were feature-dependent. Most of the missing values were coded as -1 or 0 but some of them were coded (and not listed in file DIAS Attributes) as X or XX. I converted these special codes to NaNs during pre-cleaning. 

For missing values coded as -1 or 0, I first converted them to a different code (-100) to data type-related problems and then cast them to integer NaN.

#### Assessing missing data per feature

After having all missing values converted to NaN I was able to assess which features had more than 445 000 missing. This number is half the total number of entries. As shown below, I found 9 features satisfying this requirement. They corresponded to 18% of all missing values and were all dropped.

![](figures/missing_feature.png)

#### Assessing missing data per row

In the same way certain columns can have a disproportional amount of missing values, this can also happen to groups of rows. As shown in histogram below, the distribution of missing data per row is multimodal. Most of the rows have less than 50 NaNs but some of them can have at least 180 missing features. I singled out these outliers for further assessment.

![](figures/missing_row.png)

To decide if these rows should be dropped or imputed I applied the [Kolmogorov-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test). This non-parametric test checks if two groups come from the same distribution. To apply it I selected all features that were numeric and that had no missing values in both groups. Then I compared both groups for all the features. The null hypothesis for each comparison was that both groups were identical. 

Because the test involved multiple comparisons, I also applied the very strict [Bonferroni correction](https://en.wikipedia.org/wiki/Bonferroni_correction) to the p-values. 

Results for each feature are shown below. The difference between the two groups was significant only for 8.2% of the test features. Note that this number is not a p-value and it should not be compared with the 0.05 significance level. The test was just a guide and not a proper statistical-based decision making process. In the end I decided that 8.2% was not reason enough to drop the rows.

![](figures/ks_test.png)


#### Imputing missing data

I carried out imputation separately for numeric and object variables. Numeric variables were imputed using the [Simple Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) with strategy median value. Object-type variables had their missing values replaced by the column mode using pandas [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) method. 

I also performed a few exploratory tests with the new [Iterative Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html) which trains a model to predict the ideal value. K-nearest neighbor was used as the imputing model but the results were similar to conventional Simple Imputer at a much higher computational cost. 

#### Re-encoding mixed features

After removing all NaNs and outliers, next step was to re-encode variables of mixed type. These were:

* PRAEGENDE_JUGENDJAHRE
* CAMEO_INTL_2015
* LP_LEBENSPHASE_FEIN
* LP_LEBENSPHASE_GROB

I could also have re-encoded variable PLZ8_BAUMAX but the explanatory gain was too small. Detailed descriptions of the derived variables and their levels are provided in the [notebook]().

#### One-hot encoding

So far I cleaned all data and re-encoded mixed variables. Next, I applied [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to all 13 categorical variables. Binary variables were left out.

#### Scaling

Next I applied scaling all numeric variables (including ordinals). At first I used Standard Scaler but it was too sensitive to outliers. Later I figured out that a combination of the [Robust Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html) with outlier removal greatly improved prediction results.

#### Removing outliers

This step was fundamental to improve the predictor performance in supervised learning. I removed the outliers following a three-step process:

* z-transform all scaled variables 
* mark all elements with an absolute distance greater than 4.0.
* remove all rows with at least one outlier

The threshold distance of 4.0 is actually larger than the standard inter-quartile range (IQR) distance of 3.0. I applied it because IQR was too harsh and eliminated too many rows.

I also did some eploratory tests with two native scikit-learn outlier removal methods: Isolation Forest and Local Outlier Factor. They failed to eliminate most of the outliers. Both have quite many tuning parameters because they use machine learning methods to identify the outliers. It is quite possible that I could have gotten a better result if I had properly tuned them.

### Customer Segmentation Report

The objective of the first part of the project was to reduce the dimensionality of the Azdias dataset (sample of general population of Germany) and to categorize the population in clusters. Then apply the same transformations to the Customers dataset and identify the clusters with excess customer population. By following this procedure we can identify which clusters are the most important for new customers identification in the general population.

The analysis comprised the following steps:

1. Reduce dimensionality of the general population dataset using PCA
2. Clusterize the reduced space using K-means in order to identify customer segments
3. Apply the PCA transformation defined in 1 to the customers dataset
4. Clusterize the customer reduced space and identify which clusters have a population excess

#### PCA

After being cleaned, one-hot encoded and scaled, the number of features increased to 496. Some of these features were redundant and many of them were not important at all. It is interesting, at this point, to reduce dimensionality so we can work on a leaner dataset that is more ammenable to visualization and to training.

[PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) is the most basic tool for dimensionality reduction. I applied it to reduce the number of dimensions from the original 496 to 107. This is the minimum number of dimensions needed to explain 80% of the variance. It still quite large but, as we can see in the scree plot below, only about 10 dimensions are actually individually relevant for the explained variance.

![](figures/screeplot_pca.png)

#### K-Means

Next, after reducing the number of dimensions of the data space, I applied k-means to generate clusters containing similar instances. [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) is by far the most straightforward clustering approach to this problem and it basically requires us only to define the number of clusters. It is also a greedy method so all points must be classified.

Choosing the number of clusters requires a trade-off between model size and distance to cluster center. In general the best value occurs at the inflexion point of the K-means score which, as shown in the scree plot below, was around 12 clusters for this dataset. 

One interesting point to make here is that we could have performed a more formal analysis of the ideal number of clusters using the silhouette score. However, the calculation involves a too high computational cost which prevented its application.

![](figures/screeplot_kmeans.png)

The barplot below shows how all elements of Azdias are distributed among the 12 clusters. We can see that the distribution is approximately balanced, which is a good indication that the method is consistent. 

![](figures/general_pop.png)

One of the main advantages of using PCA is that it allows us to inspect projections of a high-dimensional dataset. Because the two first components are so strong we can plot all the datapoint on this 2D space and get a good idea of how the data distributed and how the clusters were assignes. That's exactly what is shown in the plot below. 

![](figures/general_clusters.png)

Note that the data distribution has a homogeneous oval shape which is not quite separable. There are no evident spatially-separated clusters. If we look at the color map identifying different clusters, it is clear that K-Means performed a somewhat arbitrary cluster separation. Actually this method is probably not the most adequate for this dataset. [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) would be a more adqueate choice but the high computational cost prevented its application since it requires many iterations to adjust its two main parameters.

In past iterations of this project I also tried to other dimensionality reduction techniques applied after PCA: T-SNE and UMAP. Some of the results were promising but not-consistent enough to be considered for this project.

### Application to Customers dataset

The data cleaning and processing steps described above actually constitute a pipeline that can be applied to any dataset having the same features as Azdias.

Customers dataset has the same structure as Azdias with 3 extra columns (PRODUCT_GROUP, CUSTOMER_GROUP, ONLINE_PURCHASE). We drop these because they don't have any relevant information for the model training. 

If we apply the pipeline to the rest of the Customers dataset, we obtain the cluster population distribution shown in the graph below.

![](figures/customer_clusters.png)

That is a very interesting result. The customers dataset, when projected onto the clusters defined using Azdias, shows a population excess in clusters 3 and 5. This results suggests that current customers of the company share some common traits and can be singled out from the general population. If we revert this thought and look back at Azdias, all of its elements falling in clusters 3 or 5 are the ones that should be targeted by the mailout campaign. 

Another interesting consequence of this results, is that we infer some of the actual customers common traits. Looking at the PCA components weights for clusters 3 and 5 as plotted below, we note that only a few of them are relevant for the description of th cluster.

![](figures/kmeans_comps.png)

Take cluster 5. Clearly components 3, 4 and 5 are the most important. Each of these components is an axis with a positive and a negative orientation. Both are important. The positive features grow together and inversely to the negative ones.

Let's look at the most important attributes for components 3, 4 and 5.
Given a component we selected the top positive and negative attributes. These two blocks change together and inversely to one another.

Component 3: top 5 positive attributes

1. PLZ8_ANTG3: number of 6-10 family house in the PLZ8
2. KBA13_ANTG3: undefined. Probably number of 6-10 family houses in the KBA13
3. KBA13_ANTG4: undefined. Probably number of >10 family house in the KBA13
4. PLZ8_ANTG4: number of >10 family house in the PLZ8
5. KBA13_BAUMAX: undefined. Probably related to housing

Component 3: top 5 negative attributes
    
1. KBA13_AUTOQUOTE: share of cars per household within the PLZ8 
2. KBA05_ANTG1: number of 1-2 family houses in the cell
3. MOBI_REGIO: movement patterns
4. KBA13_ANTG1: number of 1-2 family houses in the KBA13
5. PLZ8_ANTG1: number of 1-2 family houses in the PLZ8

For component 0 the pattern is clear. Positive attributes are related to high-density housing and negative attributes are relate to low-density housing, possibly in the suburbs because of the movement pattern feature.

Component 4: top 5 positive attributes

1. PRAEGENDE_JUGENDJAHRE_intervall: period when individual was born
2. CJT_TYP_1: undefined
3. FINANZ_SPARER: money saver
4. CJT_TYP_2: undefined
5. SEMIO_PFLICHT: affinity indicating in what way the person is dutyfull traditional minded

Component 4: top 5 negative attributes

1. CJT_TYP_3: undefined
2. CJT_TYP_4: undefined
3. ALTERSKATEGORIE_GROB: age through prename analysis modelled on millions of first name-age-reference data
4. CJT_TYP_5: undefined
5. FINANZ_VORSORGER: financial typology: be prepared

For component 2 both positive and negative attributes are probably related to conservative-minded older individuals. Too many attribute descriptions are missing.

Component 5: top 5 positive attributes

1. GEBURTSJAHR: birth year
2. D19_VERSAND_ONLINE_QUOTE_12: amount of online transactions within all transactions in the segment mail-order
3. D19_GESAMT_ONLINE_QUOTE_12: amount of online transactions within all transactions in the complete file
4. D19_GESAMT_ANZ_24: undefined
5. D19_VERSAND_ANZ_24 undefined

Component 5: top 5 negative attributes

1. D19_GESAMT_ONLINE_DATUM: actuality of the last transaction with the complete file ONLINE
2. D19_VERSAND_DATUM: actuality of the last transaction for the segment mail-order ONLINE
3. D19_GESAMT_DATUM: actuality of the last transaction with the complete file TOTAL
4. KOMBIALTER: undefined
5. D19_KONSUMTYP_MAX: undefined

For component 3 both positive and negative attributes were related to online transactions. 

We could go deeper into this analysis using more than just the top 5 feature and maybe parsing the results using Latent Dirichlet Allocation to identify topics associated with each of the clusters. However, this is outside the scope of this work.

### Supervised Learning Model

The previous section was aimed at selecting new potential customers that would receive a mailout campaign. These were selected by Arvato which provided us with two extra datasets (mailout_train, mailout_test)containing just these individuals. Both have approximately the same size with around 43000 entries. Mailout train is the training set and it contains the same features as Azdias (366 columns) and and extra RESPONSE column indicating if the person became a customer of the company following the campaign. The other block was used to generate predictions.

As expected for this kind of study, the response classes are very imbalanced. The classes are:

* 0 - did not become customer after campaign
* 1 - became customer after campaign

Only 1.2% of the entries are of class 1.

If we look at the population distribution per cluster for response 0:

![](figures/mailout_clusters_0.png)

and for response 1:

![](figures/mailout_clusters_1.png)

we observe that they are quite similar to each other and to the customers distribution. This means that:

1. the cluster number does not convey much information for differentiation between classes 0 and 1
2. the elements that received the mailout campaign were indeed selected from the customers analysis. This is a good indication of the consistency of the method.

Another interesting visualization is to plot the data point projected on the first two PCA axes as shown below:

![](figures/mailout_response.png)

Note how the few points with reponse 1 (became clients) are embedded in the much larger homogeneous volume generated by the elements with reponse 0. Even if we plot only class 1:

![](figures/mailout_response_1.png)

we find no relevant spatial seggregation of the minority class. 

It is worthy pointing out that this is a place where T-SNE might be helpful. Some preliminary attempts were not well-succeeded but, in principle, this method could help separating the two classes by correlation.

#### Models

Class imblance is a hard problem to deal with. Our first approach to this issue is to use [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html) to balance the training set. However, this is not without risks. Some important points to keep in mind are:

* SMOTE must only be applied to training set. Predictions on Test set must use the original imbalanced classes
* if inside of a Cross-Validation loop, SMOTE must be calculated for every fold
* We applied SMOTE only to the PCA-transformed dataset because all features are numeric and real. For categorical variables SMOTENC must be used insted
* We performed oversampling of the miniority class. We could also have performed undersampling of the majority class

We first test different models using their default parameters. For each model we run 5-fold CV training and testing with and without SMOTE. The results obtained with the test set are shown below.

| Classifier        | NO SMOTE F1 | NO SMOTE AUC | SMOTE F1 | SMOTE AUC  |
| ----------------- |:-----------:|:------------:|:--------:|-----------:|
| AdaBoost          | 0.0         | 0.59         | 0.03     | 0.54       |
| Random Forests    | 0.0         | 0.51         | 0.03     | 0.55       |
| XGBoost           | 0.027       | 0.49         | 0.03     | 0.55       |

That is not very compelling. The best optimized model, obtained after running the a Grid Search routine, was an [AdaBoost Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) with learning_rate=1.0 and n_estimators=300.

Actually, running grid search with SMOTE is not easy. As we can see below, it requires the use of a SMOTE pipeline integrated with the training model. Scoring is roc_auc and we use 3-fold split instead of 5-fold to save processing time.

````python

ab_model = Pipeline([
        ('sampling', SMOTE()),
        ('ab', AdaBoostClassifier())])

parameters = {"ab__learning_rate" : [0.9, 1.0, 1.1, 2.0],
              "ab__n_estimators": [100, 200, 300, 1000]}

kf = StratifiedKFold(n_splits=3)
grid_ab = GridSearchCV(ab_model, parameters, n_jobs=16, cv=kf, scoring='roc_auc', verbose=10)

grid_ab.fit(X, y)
```

Optimized results were slightly better but far from being good:

| Classifier        | NO SMOTE F1 | NO SMOTE AUC | SMOTE F1 | SMOTE AUC  |
| ----------------- |:-----------:|:------------:|:--------:|-----------:|
| AdaBoost Optimized| 0.0         | 0.55         | 0.04     | 0.58       |

[//]:<> (If a model is used, the following should hold: The final model’s qualities — such as parameters — are evaluated in detail. Some type of analysis is used to validate the robustness of the model’s solution.)

[//]:<> (Alternatively a student may choose to answer questions with data visualizations or other means that don't involve machine learning if a different approach best helps them address their questions of interest.)

SMOTE is clearly not the best solution for this problem. After many trial and error attempts using different models, two modifications greatly improved the results:

1. Use the scale_pos_weight XGBoost parameter for compensation of class imbalance
2. Use the cleaned dataframe instead of the PCA-transformed one

The second modification was surprising. In principle we should expect that PCA contains the essence of the dataset but that was not the case. Probably the cuttoff of 80% explained variance is too low. The fact is that the only we can get beyond 0.7 AUC is to use the full dataset. As we can see below, the results were much better now:

| Classifier        | F1 | AUC |
| ----------------- |:-----------:|-----------:|
| XGBoost Optimized| 0.0         | 0.76  |

The best XGBoost model parameters after random search optimization were:

- base_score=0.5
- booster='dart'
- colsample_bylevel=1,
- colsample_bynode=1
- colsample_bytree=0.7
- gamma=0,
- learning_rate=0.01, 
- max_delta_step=0, 
- max_depth=6,
- min_child_weight=1
- n_estimators=500, 
- nthread=None, 
- objective='binary:logistic', 
- random_state=34,
- reg_alpha=0, 
- reg_lambda=1, 
- sample_pos_weight=80,
- scale_pos_weight=1

It is worthy noting that I also tried some variations of the dataset:

* add clusters as one-hot encoded features
* add the customers dataset to class 1

None of them increased the final score.

### Prediction and Kaggle competition 

Finally I used the best model to generate predictions with the Mailout_test and apply them to the Kaggle competition. The score was 0.79917, not bad but somewhat far from the first place. This result seems good in principle but in fact it is very bad. I came to this conclusion after inspecting the confusion matrix obtained for this same model:

| 14018  | 0  |
|:------:|:--:|
|  160   | 0  |

This is telling us that all minority values were misclassified. It is basically just the Naive Classifier! And that was sufficient to get a score of almost 0.8! I consider that AUC was not a good metric choice for this problem and that F1 would have been more adequate. In fact, I ran grid search again optimizing the parameters to maximize F1 instead of AUC and I obtained the following results:

| Classifier        | F1 | AUC |
| ----------------- |:-----------:|-----------:|
| XGBoost Optimized | 0.059         | 0.71  |

This model has a much more balanced confusion matrix:

| 12541  | 1477 |
|:------:|:---:|
|  237   | 135   |

The point here is that this model presented a much more balanced confusion matrix which is after all our final goal. This behavior was captured by the F1 score, which increased, and not by the AUC score, which decreased. This was the best F1 model but not the best AUC. In fact, AUC scores resemble the high scores obtained with accuracy. This is the reason why I consider that AUC is not the best metric for this problem.

## Conclusions and next steps

That was a quite interesting project, I enjoyed very much working on it. Here are the main conclusions I obtained from it:

1. the new integer NaN implemented in Pandas helps a lot with the cleaning
2. outlier removal was very important for result improvement
3. PCA and k-means are good tools for qualitative data analysis, specially if you want to use visualization to generate insights. However, for this dataset, they did not lead to better predictions
4. Using XGBoost with sample weight parameters was a better solution to class imbalance than SMOTE
5. Definitely you can get misled if you use bad metrics

We can still take a few steps to improve this model: 

1. A more thorough T-SNE parameter search after performing PCA on the Mailout Train set. If T-SNE can spatially separate the classes, the problem would be solved without need for supervised learning. 
2. Use DBSCAN instead of K-Means for cluster generation. This would probably not improve the customer segmentation analysis but it could be helpful as an extra feature for supervised learning.